In [49]:
library(mlbench)
library(caret)
library(dplyr)
library(party)
library(rpart)
library(e1071)
library(randomForest)
library(ROSE)
devtools::install_github("rstudio/keras")

config     (NA -> 0.3.1) [CRAN]
tfruns     (NA -> 1.5.0) [CRAN]
zeallot    (NA -> 0.1.0) [CRAN]
tensorflow (NA -> 2.5.0) [CRAN]


Installing 4 packages: config, tfruns, zeallot, tensorflow

‘/opt/homebrew/lib/R/4.1/site-library’의 위치에 패키지(들)을 설치합니다.
(왜냐하면 ‘lib’가 지정되지 않았기 때문입니다)



✔  checking for file ‘/private/var/folders/nn/zh17jn6d2q1c45wd419872qw0000gn/T/RtmpupTZJ7/remotes2fed458acb97/rstudio-keras-36dc8b7/DESCRIPTION’
─  preparing ‘keras’: (404ms)
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
   Removed empty directory ‘keras/man-roxygen’
─  building ‘keras_2.4.0.9000.tar.gz’
   


‘/opt/homebrew/lib/R/4.1/site-library’의 위치에 패키지(들)을 설치합니다.
(왜냐하면 ‘lib’가 지정되지 않았기 때문입니다)



In [55]:
data(BreastCancer)
BreastCancer = BreastCancer %>% select(-Id)
str(BreastCancer)

'data.frame':	699 obs. of  10 variables:
 $ Cl.thickness   : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 5 5 3 6 4 8 1 2 2 4 ...
 $ Cell.size      : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 1 4 1 8 1 10 1 1 1 2 ...
 $ Cell.shape     : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 1 4 1 8 1 10 1 2 1 1 ...
 $ Marg.adhesion  : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 1 5 1 1 3 8 1 1 1 1 ...
 $ Epith.c.size   : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 2 7 2 3 2 7 2 2 2 2 ...
 $ Bare.nuclei    : Factor w/ 10 levels "1","2","3","4",..: 1 10 2 4 1 10 10 1 1 1 ...
 $ Bl.cromatin    : Factor w/ 10 levels "1","2","3","4",..: 3 3 3 3 3 9 3 3 1 2 ...
 $ Normal.nucleoli: Factor w/ 10 levels "1","2","3","4",..: 1 2 1 7 1 7 1 1 1 1 ...
 $ Mitoses        : Factor w/ 9 levels "1","2","3","4",..: 1 1 1 1 1 1 1 1 5 1 ...
 $ Class          : Factor w/ 2 levels "benign","malignant": 1 1 1 1 1 2 1 1 1 1 ...


In [52]:
table(BreastCancer$Class)


   benign malignant 
      458       241 

위 결과를 통해 benign와 malignant 사이에 어느 정도 불균형이 존재한다고 할 수 있다.
> 불균형이 존재하면, 모형 구축시 가중치가 한쪽으로 쏠리는 문제가 발생하기 때문에 이를 보완하는 작업을 진행해야한다. 만약에 모형이 100% benign로 분류한다고 생각해보자. 그래도 60% 이상의 정확도를 확보할 수 있다. 즉 이럴 경우 매우 심각한 사태를 야기시킬 수 있기 때문에, 해당 문제를 극복할 여러 방법이 존재한다.

이러한 클래스 불균형을 해결하는 방법으로는
- 1. class가 적은 데이터에 더 큰 가중치를 주는 방법
- 2. 데이터가 더 적은 쪽으로 잘못 분류했을 때 큰 비용을 주는 방법
등이 있다. 모형을 구축해주는 함수의 parameter, loss, cost 인자로 이런 값을 지정할 수 있다.

또 다른 방법으로는 아에 훈련 데이터 셋을 조작하는 방법이다. 조작이라고 하여 매우 안좋게 볼 수 있겠지만, 이 또한 보다 좋은 모형을 만들기 위해서 사용하는 방법 중 하나이다. 이를 upsampling, downsampling, SMOTE(Synthetic Minority Technique), ROSE 등이 있다.

### Up Sampling, Down Sampling

Up Sampling은 class가 적은 데이터를 표본으로 더 많이 뽑는 방법이며, Down Sampling은 그 반대(데이터가 많은 쪽을 더 많이 뽑음)이다.
이를 사용하기 위해서는 caret package의 upSample(), downSample()이 이러한 표본 추출 방법을 지원하며 ROSE 패키지의 ovun.sample()를 통해 직접 upSample, downSample, SMOTE, ROSE 등 옵션을 선택하여 원하는 샘플링을 결정할 수 있다.

In [53]:
benign_data = BreastCancer %>% select(-Class)
subset_data = subset(BreastCancer, select=-Class)

In [54]:
# upSample(x, y) => x:독립변수, y:종속변수
upSample_data = upSample(benign_data, BreastCancer$Class)
overSample_data = ovun.sample(Class~., method = "over", N = 458*2, data = BreastCancer)$data

In [9]:
table(upSample_data$Class)


   benign malignant 
      458       458 

In [10]:
parts = createDataPartition(upSample_data$Class, p = .6)

In [11]:
str(parts)

List of 1
 $ Resample1: int [1:550] 3 4 5 6 7 9 11 13 14 16 ...


In [12]:
train = upSample_data[parts$Resample,]
test = upSample_data[-parts$Resample,]

In [13]:
res_tree = rpart(Class ~., data = train)
confusionMatrix(test$Class, predict(res_tree, newdata=test %>% select(-Class), type="class"))

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign       168        15
  malignant      7       176
                                         
               Accuracy : 0.9399         
                 95% CI : (0.9104, 0.962)
    No Information Rate : 0.5219         
    P-Value [Acc > NIR] : <2e-16         
                                         
                  Kappa : 0.8798         
                                         
 Mcnemar's Test P-Value : 0.1356         
                                         
            Sensitivity : 0.9600         
            Specificity : 0.9215         
         Pos Pred Value : 0.9180         
         Neg Pred Value : 0.9617         
             Prevalence : 0.4781         
         Detection Rate : 0.4590         
   Detection Prevalence : 0.5000         
      Balanced Accuracy : 0.9407         
                                         
       'Positive' Class : benign         
                   

In [14]:
upSample_data = downSample(benign_data, BreastCancer$Class)

In [15]:
table(upSample_data$Class)


   benign malignant 
      241       241 

In [16]:
train = upSample_data[parts$Resample,]
test = upSample_data[-parts$Resample,]

In [46]:
res_tree = rpart(Class ~., data = train)
confusionMatrix(test$Class, predict(res_tree, newdata=test %>% select(-Class), type="class"))
table(test$Class, f_res)

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign       168        15
  malignant      7       176
                                         
               Accuracy : 0.9399         
                 95% CI : (0.9104, 0.962)
    No Information Rate : 0.5219         
    P-Value [Acc > NIR] : <2e-16         
                                         
                  Kappa : 0.8798         
                                         
 Mcnemar's Test P-Value : 0.1356         
                                         
            Sensitivity : 0.9600         
            Specificity : 0.9215         
         Pos Pred Value : 0.9180         
         Neg Pred Value : 0.9617         
             Prevalence : 0.4781         
         Detection Rate : 0.4590         
   Detection Prevalence : 0.5000         
      Balanced Accuracy : 0.9407         
                                         
       'Positive' Class : benign         
                   

           f_res
            benign malignant
  benign       166        17
  malignant      2       181

In [19]:
table(overSample_data$Class)


   benign malignant 
      444       472 

In [20]:
str(overSample_data)

'data.frame':	916 obs. of  10 variables:
 $ Cl.thickness   : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 5 5 3 6 4 1 2 2 4 1 ...
 $ Cell.size      : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 1 4 1 8 1 1 1 1 2 1 ...
 $ Cell.shape     : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 1 4 1 8 1 1 2 1 1 1 ...
 $ Marg.adhesion  : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 1 5 1 1 3 1 1 1 1 1 ...
 $ Epith.c.size   : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 2 7 2 3 2 2 2 2 2 1 ...
 $ Bare.nuclei    : Factor w/ 10 levels "1","2","3","4",..: 1 10 2 4 1 10 1 1 1 1 ...
 $ Bl.cromatin    : Factor w/ 10 levels "1","2","3","4",..: 3 3 3 3 3 3 3 1 2 3 ...
 $ Normal.nucleoli: Factor w/ 10 levels "1","2","3","4",..: 1 2 1 7 1 1 1 1 1 1 ...
 $ Mitoses        : Factor w/ 9 levels "1","2","3","4",..: 1 1 1 1 1 1 1 5 1 1 ...
 $ Class          : Factor w/ 2 levels "benign","malignant": 1 1 1 1 1 1 1 1 1 1 ...


In [37]:
res_overSample = rpart(Class ~ ., data=overSample_data)

In [40]:
pred_over = predict(res_overSample, newdata = test)
length(pred_over[,2])
nrow(test)

[1] 366

[1] 366

In [41]:
roc.curve(test$Class, pred_over[,2], plot=FALSE)

Area under the curve (AUC): 0.954

In [43]:
confusionMatrix(test$Class, f_res, type="class")

ERROR: Error in table(data, reference, dnn = dnn, ...): 모든 인자들은 반드시 같은 길이를 가져야 합니다


In [26]:
pred_over[,2]

1           2           8          10          12          15 
0.007575758 0.956709957 0.007575758 0.007575758 0.007575758 0.007575758 
         23          29          30          36          37          39 
0.007575758 0.007575758 0.007575758 0.007575758 0.956709957 0.007575758 
         42          45          47          54          56          57 
0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 
         58          59          61          64          65          67 
0.007575758 0.833333333 0.000000000 0.007575758 0.007575758 0.007575758 
         70          72          74          78          80          81 
0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 0.000000000 
         83          85          86          87          91          94 
0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 
        101         103         104         105         108         112 
0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 
        113         115         121         123         125         128 
0.956709957 0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 
        129         130         140         142         143         147 
0.007575758 0.956709957 0.007575758 0.956709957 0.007575758 0.007575758 
        152         155         156         158         159         161 
0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 0.956709957 
        164         166         170         171         175         176 
0.007575758 0.007575758 0.007575758 0.007575758 0.956709957 0.007575758 
        178         179         180         186         187         192 
0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 
        194         196         198         202         203         205 
0.956709957 0.007575758 0.956709957 0.007575758 0.000000000 0.007575758 
        206         208         209         210         211         212 
0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 0.956709957 
        217         219         220         224         225         227 
0.000000000 0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 
        228         232         237         238         247         248 
0.007575758 0.000000000 0.833333333 0.007575758 0.956709957 0.007575758 
        249         250         253         255         256         260 
0.007575758 0.956709957 0.000000000 0.007575758 0.007575758 0.007575758 
        261         262         266         267         272         274 
0.894736842 0.007575758 0.007575758 0.007575758 0.000000000 0.007575758 
        275         282         285         287         288         290 
0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 
        291         293         296         300         301         302 
0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 
        303         304         308         311         312         313 
0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 
        316         322         328         329         335         336 
0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 
        337         338         340         343         344         346 
0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 
        350         351         352         354         355         358 
0.000000000 0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 
        361         362         363         365         366         367 
0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 
        369         370         371         372         375         378 
0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 
        380         382         386         388         393         395 
0.007575758 0.007575758 0.007575758 0.007575758 0.007575758 0.956709957 
        396         398         400         403         410         

In [27]:
f_res = c()
for(i in pred_over[,2]) {
    if(i >= 0.5) f_res <- c(f_res, "malignant") else
        f_res <- c(f_res, "benign")
}
f_res = factor(f_res)

In [29]:
summary(f_res)

benign malignant 
      168       198

In [30]:
summary(test$Class)

benign malignant 
      183       183

In [32]:
length(test$Class) ; length(f_res)

[1] 366

[1] 366